In [2]:
import ast

In [3]:
def calculate_sum_of_three_elements(a, b, c):
    return a + b + c

def calculate_sum_of_three_elements2(a: int, b: int, c: int):
    a += b
    a += c
    return a

def calculate_sum_of_two_elements(a: int, b: int) -> int:
    return a + b

def calculate_sum_of_three_elements3(a: int, b: int, c: int) -> int:
    return calculate_sum_of_two_elements(calculate_sum_of_three_elements(a, b), c)

def calculate_sum_of_three_elements4(a, b, c):
    return calculate_sum_of_two_elements(a, c) + b

def calculate_sum_of_three_elements5(a, b, c):
    return sum([a, b, c])

In [4]:
csotes = """
def calculate_sum_of_three_elements(a, b, c):
    return a + b + c

def calculate_sum_of_three_elements2(a: int, b: int, c: int):
    a += b
    a += c
    return a

def calculate_sum_of_two_elements(a: int, b: int) -> int:
    return a + b

def calculate_sum_of_three_elements3(a: int, b: int, c: int) -> int:
    return calculate_sum_of_two_elements(calculate_sum_of_three_elements(a, b), c)

def calculate_sum_of_three_elements4(a, b, c):
    return calculate_sum_of_two_elements(a, c) + b

def calculate_sum_of_three_elements5(a, b, c):
    return sum([a, b, c])
""".strip().split("\n\n")

In [5]:
csotes

['def calculate_sum_of_three_elements(a, b, c):\n    return a + b + c',
 'def calculate_sum_of_three_elements2(a: int, b: int, c: int):\n    a += b\n    a += c\n    return a',
 'def calculate_sum_of_two_elements(a: int, b: int) -> int:\n    return a + b',
 'def calculate_sum_of_three_elements3(a: int, b: int, c: int) -> int:\n    return calculate_sum_of_two_elements(calculate_sum_of_three_elements(a, b), c)',
 'def calculate_sum_of_three_elements4(a, b, c):\n    return calculate_sum_of_two_elements(a, c) + b',
 'def calculate_sum_of_three_elements5(a, b, c):\n    return sum([a, b, c])']

In [6]:
trees = [ast.parse(csote) for csote in csotes]

In [7]:
for tree in trees:
    print(ast.dump(tree, indent=4))
    print("\n\n")

Module(
    body=[
        FunctionDef(
            name='calculate_sum_of_three_elements',
            args=arguments(
                posonlyargs=[],
                args=[
                    arg(arg='a'),
                    arg(arg='b'),
                    arg(arg='c')],
                kwonlyargs=[],
                kw_defaults=[],
                defaults=[]),
            body=[
                Return(
                    value=BinOp(
                        left=BinOp(
                            left=Name(id='a', ctx=Load()),
                            op=Add(),
                            right=Name(id='b', ctx=Load())),
                        op=Add(),
                        right=Name(id='c', ctx=Load())))],
            decorator_list=[])],
    type_ignores=[])



Module(
    body=[
        FunctionDef(
            name='calculate_sum_of_three_elements2',
            args=arguments(
                posonlyargs=[],
                args=[
                    arg(
        

In [8]:
def ast_to_dict(node):
    """Convert an AST node to a dictionary."""
    if isinstance(node, ast.AST):
        fields = {}
        for name, value in ast.iter_fields(node):
            fields[name] = ast_to_dict(value)
        return {
            "type": type(node).__name__,
            "fields": fields,
        }
    elif isinstance(node, list):
        return [ast_to_dict(x) for x in node]
    else:
        return node

In [9]:
ast_to_dict(trees[0])

{'type': 'Module',
 'fields': {'body': [{'type': 'FunctionDef',
    'fields': {'name': 'calculate_sum_of_three_elements',
     'args': {'type': 'arguments',
      'fields': {'posonlyargs': [],
       'args': [{'type': 'arg',
         'fields': {'arg': 'a', 'annotation': None, 'type_comment': None}},
        {'type': 'arg',
         'fields': {'arg': 'b', 'annotation': None, 'type_comment': None}},
        {'type': 'arg',
         'fields': {'arg': 'c', 'annotation': None, 'type_comment': None}}],
       'vararg': None,
       'kwonlyargs': [],
       'kw_defaults': [],
       'kwarg': None,
       'defaults': []}},
     'body': [{'type': 'Return',
       'fields': {'value': {'type': 'BinOp',
         'fields': {'left': {'type': 'BinOp',
           'fields': {'left': {'type': 'Name',
             'fields': {'id': 'a', 'ctx': {'type': 'Load', 'fields': {}}}},
            'op': {'type': 'Add', 'fields': {}},
            'right': {'type': 'Name',
             'fields': {'id': 'b', 'ctx': {

In [10]:
class RenameVars(ast.NodeTransformer):
    def __init__(self):
        self.arg_num = 1
        self.var_num = 1
        self.var_map = {}
        
    def get_new_arg_name(self):
        name = f"arg{self.arg_num}"
        self.arg_num += 1
        return name
    
    def get_new_var_name(self):
        name = f"var{self.var_num}"
        self.var_num += 1
        return name
    
    def visit_FunctionDef(self, node):
        # Rename function arguments
        for arg in node.args.args:
            arg.arg = self.get_new_arg_name()

        # Rename variables in function body
        for stmt in node.body:
            if isinstance(stmt, ast.Assign):
                for target in stmt.targets:
                    if isinstance(target, ast.Name):
                        target.id = self.get_new_var_name()
                        
        return node

    def visit_Name(self, node):
        # Rename variables
        if isinstance(node.ctx, ast.Load):
            old_name = node.id
            if old_name in self.var_map:
                node.id = self.var_map[old_name]
            else:
                new_name = self.get_new_var_name()
                self.var_map[old_name] = new_name
                node.id = new_name
        return node

def rename_vars(code):
    ast_tree = ast.parse(code)
    renamer = RenameVars()
    renamed_ast = renamer.visit(ast_tree)
#     return ast.unparse(renamed_ast)
    return ast.dump(renamed_ast, indent=4)

In [11]:
code = """
def foo(x):
    y = 1 + x
    z = foo(y)
    return z
"""

print(rename_vars(code))
print(ast.dump(ast.parse(code), indent=4))

Module(
    body=[
        FunctionDef(
            name='foo',
            args=arguments(
                posonlyargs=[],
                args=[
                    arg(arg='arg1')],
                kwonlyargs=[],
                kw_defaults=[],
                defaults=[]),
            body=[
                Assign(
                    targets=[
                        Name(id='var1', ctx=Store())],
                    value=BinOp(
                        left=Constant(value=1),
                        op=Add(),
                        right=Name(id='x', ctx=Load()))),
                Assign(
                    targets=[
                        Name(id='var2', ctx=Store())],
                    value=Call(
                        func=Name(id='foo', ctx=Load()),
                        args=[
                            Name(id='y', ctx=Load())],
                        keywords=[])),
                Return(
                    value=Name(id='z', ctx=Load()))],
            decorato

In [12]:
code = """
def foo(x):
    y = 1 + x
    z = foo(y)
    return z
"""
ast_tree = ast.parse(code)
renamer = RenameVars()
renamed_ast = renamer.visit(ast_tree)

In [13]:
print(ast.dump(renamed_ast, indent=4))

Module(
    body=[
        FunctionDef(
            name='foo',
            args=arguments(
                posonlyargs=[],
                args=[
                    arg(arg='arg1')],
                kwonlyargs=[],
                kw_defaults=[],
                defaults=[]),
            body=[
                Assign(
                    targets=[
                        Name(id='var1', ctx=Store())],
                    value=BinOp(
                        left=Constant(value=1),
                        op=Add(),
                        right=Name(id='x', ctx=Load()))),
                Assign(
                    targets=[
                        Name(id='var2', ctx=Store())],
                    value=Call(
                        func=Name(id='foo', ctx=Load()),
                        args=[
                            Name(id='y', ctx=Load())],
                        keywords=[])),
                Return(
                    value=Name(id='z', ctx=Load()))],
            decorato

In [14]:
import astor

print(astor.to_source(renamed_ast))

def foo(arg1):
    var1 = 1 + x
    var2 = foo(y)
    return z



In [15]:
import ast

# Example code to normalize
code = '''
def foo(x: int, y: int) -> int:
    z = x + y
    if z > 10:
        return z
    else:
        return x * y
'''

# Parse the code into an AST
tree = ast.parse(code)

# Define a function to generate new variable names
var_counter = 1
def new_var():
    global var_counter
    name = f'var{var_counter}'
    var_counter += 1
    return name

# Define a function to rename variables
def rename_variables(node):
    global var_counter
    if isinstance(node, ast.Name):
        node.id = new_var()
    elif isinstance(node, ast.arg):
        node.arg = f'arg{var_counter}'
        var_counter += 1
    for child in ast.iter_child_nodes(node):
        rename_variables(child)

# Define a function to remove function typings
def remove_typings(node):
    if isinstance(node, ast.FunctionDef):
        node.returns = None
        for arg in node.args.args:
            arg.annotation = None
    for child in ast.iter_child_nodes(node):
        remove_typings(child)

# Rename variables and remove function typings
rename_variables(tree)
remove_typings(tree)

# Define a function to replace variables with arguments if they are the same
def replace_variables_with_arguments(node):
    if isinstance(node, ast.Name):
        if node.id.startswith('var') and node.id[3:].isdigit():
            index = int(node.id[3:]) - 1
            if isinstance(node.parent, ast.FunctionDef):
                if index < len(node.parent.args.args):
                    arg = node.parent.args.args[index]
                    node.id = arg.arg
            elif isinstance(node.parent, ast.Assign):
                targets = node.parent.targets
                if index < len(targets):
                    target = targets[index]
                    if isinstance(target, ast.Name):
                        node.id = target.id
    for child in ast.iter_child_nodes(node):
        child.parent = node
        replace_variables_with_arguments(child)

# Replace variables with arguments if they are the same
replace_variables_with_arguments(tree)

# Normalize the AST
ast.fix_missing_locations(tree)

# Print the normalized AST
print(ast.unparse(tree))

def foo(arg1, arg3):
    var5 = var6 + var7
    if var8 > 10:
        return var9
    else:
        return var10 * var11


In [16]:
import ast
import inspect

def foo(x: int, y: int) -> int:
    z = x + y
    if z > 10:
        return z
    else:
        return x * y

def normalize_function(code):
    # Parse the code into an AST
    tree = ast.parse(code)

    # Rename variables and arguments
    def rename_variables_and_arguments(node):
        if isinstance(node, ast.Name):
            if node.id.startswith('arg') and node.id[3:].isdigit():
                index = int(node.id[3:]) - 1
                node.id = f'arg{index + 1}'
            elif node.id.startswith('var') and node.id[3:].isdigit():
                index = int(node.id[3:]) - 1
                node.id = f'var{index + 1}'
        for child in ast.iter_child_nodes(node):
            child.parent = node
            rename_variables_and_arguments(child)

    rename_variables_and_arguments(tree)

    # Remove function typings
    def remove_function_typings(node):
        if isinstance(node, ast.AnnAssign):
            node.annotation = None
        elif isinstance(node, ast.arg):
            node.annotation = None
        for child in ast.iter_child_nodes(node):
            child.parent = node
            remove_function_typings(child)

    remove_function_typings(tree)

    # Replace variables with arguments if they are the same
    def replace_variables_with_arguments(node):
        if isinstance(node, ast.Name):
            if node.id.startswith('var') and node.id[3:].isdigit():
                index = int(node.id[3:]) - 1
                if isinstance(node.parent, ast.FunctionDef):
                    if index < len(node.parent.args.args):
                        arg = node.parent.args.args[index]
                        node.id = arg.arg
                elif isinstance(node.parent, ast.Assign):
                    targets = node.parent.targets
                    if index < len(targets):
                        target = targets[index]
                        if isinstance(target, ast.Name):
                            node.id = target.id
        for child in ast.iter_child_nodes(node):
            child.parent = node
            replace_variables_with_arguments(child)

    replace_variables_with_arguments(tree)

    # Generate the new code
    new_code = ast.unparse(tree)

    return new_code

# Example usage
code = inspect.getsource(foo)
new_code = normalize_function(code)
print(new_code)

def foo(x, y) -> int:
    z = x + y
    if z > 10:
        return z
    else:
        return x * y


In [17]:
def foo(x: int, y: int) -> int:
    z = x + y
    if z > 10:
        return z
    else:
        return x * y

In [18]:
import inspect

In [19]:
inspect.getsource(foo)

'def foo(x: int, y: int) -> int:\n    z = x + y\n    if z > 10:\n        return z\n    else:\n        return x * y\n'

In [20]:
import ast
import inspect

def simplify(func):
    # Parse the function code into an abstract syntax tree (AST)
    tree = ast.parse(inspect.getsource(func))

    # Define a visitor class to traverse the AST and replace method calls with the underlying atomic operations
    class MethodCallVisitor(ast.NodeTransformer):
        def visit_Assign(self, node):
            # If the function being called is a method, replace the call with the method's body with parameters renamed
            if isinstance(node.func, ast.Attribute):
                method_name = node.func.attr
                for statement in func.__code__.co_consts:
                    if isinstance(statement, type((lambda: 0).__code__)):
                        if statement.co_name == method_name:
                            # Replace the method call with the body of the method with renamed parameters
                            method_ast = ast.parse(inspect.getsource(statement))
                            method_params = method_ast.body[0].args.args
                            method_args = node.args
                            for i in range(len(method_params)):
                                method_ast.body[0].args.args[i].arg = node.args[i].id
                            ast.copy_location(method_ast, node)
                            ast.fix_missing_locations(method_ast)
                            return method_ast.body[0]

            # Otherwise, visit the node's children
            return self.generic_visit(node)

    # Traverse the AST with the MethodCallVisitor
    transformer = MethodCallVisitor()
    new_tree = transformer.visit(tree)

    # Convert the AST back into Python code as a string
    return ast.unparse(new_tree)

In [21]:
action1 = lambda a: a + 2
action2 = lambda b: b + 3
action3 = lambda c: c - 4
action4 = lambda d: 2 * d
action5 = lambda e: e ** 2

def method1(a, b, c):
    action1(a)
    action2(b)
    action3(c)

def method2(d, e):
    action4(d)
    action5(e)

def method_complex(x, y, z, u, v):
    method1(x, y, z)
    method2(u, v)
    
# Add some additional calls on the same line to test the updated implementation
def method_complex2(x, y, z, u, v):
    method1(x, y, z); method2(u, v)
    method1(x, y, z); method2(u, v)

In [22]:
print(simplify(method_complex2))

def method_complex2(x, y, z, u, v):
    method1(x, y, z)
    method2(u, v)
    method1(x, y, z)
    method2(u, v)


In [23]:
class MethodCallVisitor(ast.NodeTransformer):
    def visit_Assign(self, node: ast.Assign):
        if not isinstance(node.value, ast.Call):
            return node
        new_node = []
        func: ast.Call = node.value
        funcDef: ast.FunctionDef = ast.parse(inspect.getsource(eval(func.func.id))).body[0]
        
        assign_left = [ast.Tuple([ast.Name(arg.arg) for arg in funcDef.args.args], ast.Store())]
        assign_right = ast.Tuple(func.args, ast.Load())
        
        new_node.append(ast.Assign(assign_left, assign_right, lineno=node.lineno))
        new_node.extend([i for i in funcDef.body if not isinstance(i, ast.Return)])
        
        returned: ast.Return = [i for i in funcDef.body if isinstance(i, ast.Return)][0]
        new_node.append(ast.Assign(node.targets, returned.value, lineno=node.lineno))
        
        return new_node

In [24]:
# tree = ast.parse(inspect.getsource(method_complex2))
# print(ast.dump(tree, indent=4))

In [25]:
mcv = MethodCallVisitor()
new_tree = mcv.visit(tree)
print(ast.dump(new_tree, indent=4))

Module(
    body=[
        FunctionDef(
            name='foo',
            args=arguments(
                posonlyargs=[],
                args=[
                    arg(arg='arg1'),
                    arg(arg='arg3')],
                kwonlyargs=[],
                kw_defaults=[],
                defaults=[]),
            body=[
                Assign(
                    targets=[
                        Name(id='var5', ctx=Store())],
                    value=BinOp(
                        left=Name(id='var6', ctx=Load()),
                        op=Add(),
                        right=Name(id='var7', ctx=Load()))),
                If(
                    test=Compare(
                        left=Name(id='var8', ctx=Load()),
                        ops=[
                            Gt()],
                        comparators=[
                            Constant(value=10)]),
                    body=[
                        Return(
                            value=Name(id='var9

In [26]:
inspect.getsource(eval("method1"))

'def method1(a, b, c):\n    action1(a)\n    action2(b)\n    action3(c)\n'

In [27]:
def print_ast(method):
    print(ast.dump(ast.parse(inspect.getsource(method)), indent=4))
    
def get_ast(method):
    return ast.parse(inspect.getsource(method))

In [28]:
def foo1(arg1, arg2, arg3):
    arg1 += arg2
    arg3 /= arg1
    arg2 += arg3
    arg1 /= arg2
    arg3 += arg1
    var1 = arg1 + arg2 + arg3
    var2 = arg1 - arg2 - arg3
    var3 = var1 - var2
    return var3

In [29]:
print_ast(foo1)

Module(
    body=[
        FunctionDef(
            name='foo1',
            args=arguments(
                posonlyargs=[],
                args=[
                    arg(arg='arg1'),
                    arg(arg='arg2'),
                    arg(arg='arg3')],
                kwonlyargs=[],
                kw_defaults=[],
                defaults=[]),
            body=[
                AugAssign(
                    target=Name(id='arg1', ctx=Store()),
                    op=Add(),
                    value=Name(id='arg2', ctx=Load())),
                AugAssign(
                    target=Name(id='arg3', ctx=Store()),
                    op=Div(),
                    value=Name(id='arg1', ctx=Load())),
                AugAssign(
                    target=Name(id='arg2', ctx=Store()),
                    op=Add(),
                    value=Name(id='arg3', ctx=Load())),
                AugAssign(
                    target=Name(id='arg1', ctx=Store()),
                    op=Div(),
    

In [30]:
def any_function1(a1, b1, c1):
    a1 += b1
    c1 /= a1
    b1 += c1
    a1 /= b1
    c1 += a1
    return a1, b1, c1

def any_function2(a2, b2, c2):
    d2 = a2 + b2 + c2
    e2 = a2 - b2 - c2
    return d2, e2

def any_function3(d3, e3):
    f3 = d3 - e3
    return f3

def common_function(a, b, c):
    a, b, c = any_function1(a, b, c)
    d, e = any_function2(a, b, c)
    f = any_function3(d, e)
    return f

In [31]:
print_ast(any_function1)

Module(
    body=[
        FunctionDef(
            name='any_function1',
            args=arguments(
                posonlyargs=[],
                args=[
                    arg(arg='a1'),
                    arg(arg='b1'),
                    arg(arg='c1')],
                kwonlyargs=[],
                kw_defaults=[],
                defaults=[]),
            body=[
                AugAssign(
                    target=Name(id='a1', ctx=Store()),
                    op=Add(),
                    value=Name(id='b1', ctx=Load())),
                AugAssign(
                    target=Name(id='c1', ctx=Store()),
                    op=Div(),
                    value=Name(id='a1', ctx=Load())),
                AugAssign(
                    target=Name(id='b1', ctx=Store()),
                    op=Add(),
                    value=Name(id='c1', ctx=Load())),
                AugAssign(
                    target=Name(id='a1', ctx=Store()),
                    op=Div(),
               

In [32]:
mcv = MethodCallVisitor()
tree = get_ast(common_function)

print(ast.unparse(mcv.visit(tree)))

def common_function(a, b, c):
    (a1, b1, c1) = (a, b, c)
    a1 += b1
    c1 /= a1
    b1 += c1
    a1 /= b1
    c1 += a1
    (a, b, c) = (a1, b1, c1)
    (a2, b2, c2) = (a, b, c)
    d2 = a2 + b2 + c2
    e2 = a2 - b2 - c2
    (d, e) = (d2, e2)
    (d3, e3) = (d, e)
    f3 = d3 - e3
    f = f3
    return f


In [33]:
def common_function(a, b, c):
    (a1, b1, c1) = (a, b, c)
    a1 += b1
    c1 /= a1
    b1 += c1
    a1 /= b1
    c1 += a1
    (a, b, c) = (a1, b1, c1)
    (a2, b2, c2) = (a, b, c)
    d2 = a2 + b2 + c2
    e2 = a2 - b2 - c2
    (d, e) = (d2, e2)
    (d3, e3) = (d, e)
    f3 = d3 - e3
    f = f3
    return f

In [34]:
print(ast.unparse(get_ast(common_function)))

def common_function(a, b, c):
    (a1, b1, c1) = (a, b, c)
    a1 += b1
    c1 /= a1
    b1 += c1
    a1 /= b1
    c1 += a1
    (a, b, c) = (a1, b1, c1)
    (a2, b2, c2) = (a, b, c)
    d2 = a2 + b2 + c2
    e2 = a2 - b2 - c2
    (d, e) = (d2, e2)
    (d3, e3) = (d, e)
    f3 = d3 - e3
    f = f3
    return f


In [93]:
def function(a, b, c):
    a, b, c = a, b, c
    a, b, c = a, b, c
    d, e = d, e
    return f

In [98]:
print(ast.dump(ast.parse(
"""
def function(a, b, c):
    a, b, c = a, b, c
    a, b, c = a, b, c
    d, e = d, e
    return f
"""
), indent=4))

Module(
    body=[
        FunctionDef(
            name='function',
            args=arguments(
                posonlyargs=[],
                args=[
                    arg(arg='a'),
                    arg(arg='b'),
                    arg(arg='c')],
                kwonlyargs=[],
                kw_defaults=[],
                defaults=[]),
            body=[
                Assign(
                    targets=[
                        Tuple(
                            elts=[
                                Name(id='a', ctx=Store()),
                                Name(id='b', ctx=Store()),
                                Name(id='c', ctx=Store())],
                            ctx=Store())],
                    value=Tuple(
                        elts=[
                            Name(id='a', ctx=Load()),
                            Name(id='b', ctx=Load()),
                            Name(id='c', ctx=Load())],
                        ctx=Load())),
                Assign(
   

In [106]:
ast.unparse(ast.parse("a = 5; b = 7"))

'a = 5\nb = 7'

In [1]:
class VariableRenamer(ast.NodeTransformer):
    name_map = {}
    expr_num = 0

    def visit_Name(self, node):
        if isinstance(node.ctx, ast.Load):
            # if the variable is being loaded, check if it has been renamed already
            if node.id in self.name_map:
                node.id = self.name_map[node.id]
            else:
                # generate a new variable name and add it to the map
                new_name = f"var{self.expr_num}"
                self.expr_num += 1
                self.name_map[node.id] = new_name
                node.id = new_name

        return node

NameError: name 'ast' is not defined